# Aggregating Daily Traffic Volume for the City of Chicago
Here, we will be importing traffic volume data for Chicago and aggregating it by Community Area

## Importing the necessary packages

In [ ]:
library(sp)
library(rgdal)
library(rgeos)
library(DataCombine)
library(leaflet)
library(tmap)
library(dplyr)

rgdal: version: 1.2-16, (SVN revision 701)
 Geospatial Data Abstraction Library extensions to R successfully loaded
 Loaded GDAL runtime: GDAL 2.1.3, released 2017/20/01
 Path to GDAL shared files: /Library/Frameworks/R.framework/Versions/3.4/Resources/library/rgdal/gdal
 GDAL binary built with GEOS: FALSE 
 Loaded PROJ.4 runtime: Rel. 4.9.3, 15 August 2016, [PJ_VERSION: 493]
 Path to PROJ.4 shared files: /Library/Frameworks/R.framework/Versions/3.4/Resources/library/rgdal/proj
 Linking to sp version: 1.2-5 
rgeos version: 0.3-26, (SVN revision 560)
 GEOS runtime version: 3.6.1-CAPI-1.10.1 r0 
 Linking to sp version: 1.2-5 
 Polygon checking: TRUE 

Warning message:
“package ‘leaflet’ was built under R version 3.4.4”Warning message:
“package ‘tmap’ was built under R version 3.4.4”

## Importing the Traffic Data

The City of Chicago Open Data Portal provides information on Average Daily Traffic Counts, a "close approximation to the actual number of vehicles passing through a given location on an average weekday". The counts are done approximately once every ten years, but the most recent count was conducted in 2006. This count consists of point data for 1279 unique locations. The data is available for viewing and download [here](https://data.cityofchicago.org/Transportation/Average-Daily-Traffic-Counts/pfsx-4n4m/data). For the purposes of this data collection, we will download the Average Daily Traffic Count data was downloaded as a CSV (Average_Daily_Traffic_Counts.csv). 

In [ ]:
Traffic <- read.csv("Average_Daily_Traffic_Counts.csv")

head(Traffic)

As we can see, the traffic count dataset contains information on Address, Date, Total Volume, Directional Volume, and Coordinates

## Converting the Traffic Data to a Spatial Object

In [ ]:
#Create a spatial object by assigning Lat/Lon coordinates to Traffic 
coordinates(Traffic) <- Traffic[,c("Longitude", "Latitude")]

In [ ]:
#Set CRS of Traffic data to WSG84
proj4string(Traffic) <- CRS("+init=epsg:4326")

## Importing the Chicago Map with Community Area Boundaries Shapefile

The Chicago Open Data Portal provides a map of the current boundaries for each of the 77 Community Areas in city. This map can be downloaded [here](https://data.cityofchicago.org/Facilities-Geographic-Boundaries/Boundaries-Community-Areas-current-/cauq-8yn6/data) as a Shapefile (Chi_Boundaries.shp) which we will subsequently import.

In [ ]:
Chi.Map <- readOGR("Chi_Boundaries")

In [ ]:
#Transform projection of Chicago Map to WSG84 to match Traffic data
Chi.Map <- spTransform(Chi.Map ,CRS("+init=epsg:4326"))

## Spatially Overlaying Traffic Point Data on Community Area Polygons

In order to determine which Community Area (CA) each traffic node is in, we will overlay the point data on the map of the Chicago Community Areas 

In [ ]:
#Returns a vector with length = # of Traffic points and community column with information on Community Area
PiP <- over(Traffic, Chi.Map)

In [ ]:
#Create new dataframe with the variables that we will need (Lat/Lon, Volume, Community Area)
CA.Traffic <- data.frame(PiP$community, Traffic$Latitude, Traffic$Longitude, Traffic$Total.Passing.Vehicle.Volume)
#Rename columns for readability
colnames(CA.Traffic) <- c("Community_Area", "Latitude", "Longitude", "Traffic_Volume")


## Filling in Missing Information

We must first determine whether or not there are any elements of CA.Traffic for which the Community_Area variable is NA

In [ ]:
summary(CA.Traffic)

In [ ]:
#Create dataframe of all elements with NA values for Community Area
CA.NA <- CA.Traffic[which(is.na(CA.Traffic$Community_Area)),]

In [ ]:
summary(CA.NA)

From the summary, we see that there are 13 traffic nodes for which the match function was unable to assign a Community Area. These nodes are located just over the edge of the Community Area map and should, therefore, be included with the totals for the nearest community area. We must first create a spatial object from the NA dataframe.

In [ ]:
#Assign coordinates and projection to NAs
coordinates(CA.NA) <- CA.NA[,c("Longitude", "Latitude")]
proj4string(CA.NA) <- CRS("+init=epsg:4326")

We need to now change the projection system of both the NAs and the Chicago base map to UTM so that we can properly use the rgeos function gDistance to determine the closest Community Area to a given point

In [ ]:
#Change projections for both NAs and Community Area Map to UTM Zone 16
CA.NA <- spTransform(CA.NA,CRS("+init=epsbg:26916"))
Chi.Map <- spTransform(Chi.Map ,CRS("+init=epsg:26916"))

We will now loop through the NAs and find the minimum value for gDistance between the point and a Community Area.

In [ ]:
#Container for Nearest Community Areas
n <- length(CA.NA)
Nearest.CA <- character(n)

#Iterate through finding the nearest Community Area for each NA
for (i in seq_along(Nearest.CA)) {
  Nearest.CA[i] <- as.character(Chi.Map$community[which.min(gDistance(CA.NA[i,], Chi.Map, byid = TRUE))])
  print(Nearest.CA[i])
}

Now that we have a vector of the correct Community Areas to assign each point, we need to replace the NAs in the original CA.Traffic dataframe with these vaues.

In [ ]:
#Create counter variable for NA's
j = 1

#Check if value is NA and replace with nearest CA if it is
for(i in 1:nrow(CA.Traffic)) {
  if(is.na(CA.Traffic$Community_Area)[i]) {
    CA.Traffic$Community_Area[i] = Nearest.CA[j]
    j = j + 1
  }
}

In [ ]:
#Revert CRS to WSG84 for Chicago Map
Chi.Map <- spTransform(Chi.Map ,CRS("+init=epsg:4326"))

## Aggregate Traffic by Community Area

Now that we have determined the Community Area for each traffic point, it is time to calculate the total traffic volume for each Community Area

In [ ]:
#Group Traffic data by Community Area and total up volume for each one
CA.Agg <- CA.Traffic %>% 
  group_by(Community_Area) %>%
  summarize(sum(Traffic_Volume))

colnames(CA.Agg) <- c("community", "Traffic_Volume")

nrow(CA.Agg)

As stated above, Chicago has 77 Community Areas, but only 76 are in the new dataframe. It appears that the Community Area of Burnside does not show up because no traffic points are located there. We will manually add it in and assign it a volume of 0

In [ ]:
Burnside <- c("BURNSIDE", 0)
CA.Agg <- InsertRow(CA.Agg, Burnside, RowNum = 13)

Now that we have our Community Area aggregates, we want to add the traffic volume to our Spatial Polygon map of the Communtiy Areas so that it is attached to the geographical information therein contained

In [ ]:
#first convert traffic volume to integer
CA.Agg$Traffic_Volume <- as.integer(CA.Agg$Traffic_Volume)

#Join the two dataframes
Chi.Map@data <- left_join(Chi.Map@data, CA.Agg)

In order to make sure that the join had the desired effect, let's create a quick choropleth map of the traffic volume aggregates by Community Area

In [ ]:
tm_shape(Chi.Map) +
  tm_fill("Traffic_Volume", style = "jenks", n = 7, palette = "Reds") + 
  tm_borders(alpha = 0.5)

## Exporting the Data

We want to export two shapefiles: 
1. The traffic volume point data with the additional information of the Community Area of each point
2. The Chicago Community Area map with the additional information of the aggregate traffic volume by Community Area

First, we must make a spatial dataframe out of CA.Traffic, then we can create the shapefiles 

In [ ]:
#Turn CA.Traffic into spatial points data frame
coordinates(CA.Traffic) <- CA.Traffic[,c("Longitude", "Latitude")]
proj4string(CA.Traffic) <- CRS("+init=epsg:4326")
CA.Traffic <- spTransform(CA.Traffic,CRS("+init=epsg:4326"))

#Create the two shapefiles
writeOGR(CA.Traffic,".","Traffic_Points_with_Community_Areas",driver="ESRI Shapefile")
writeOGR(Chi.Map,".","Community_Areas_with_Traffic_Volumes",driver="ESRI Shapefile")